In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 15:51:16 WARN Utils: Your hostname, codespaces-225dfc resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/04 15:51:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 15:51:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!ls

04-pyspark.ipynb  download_data.sh	      head.csv
DataType	  fhvhv			      taxi+_zone_lookup.csv
Untitled.ipynb	  fhvhv_tripdata_2021-01.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [5]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
!head -n 1001 fhv_tripdata_2019-10.csv > head_homework.csv

Have a overview of data type

In [10]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [11]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [12]:
import pandas as pd

In [13]:
df_pandas = pd.read_csv('head_homework.csv')

In [14]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

/home/codespace/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/codespace/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

Seems that the schema inference function does not work here due to mixed data types (I guess it's due to missing values)

In [17]:
filtered_df = df_pandas[df_pandas['Affiliated_base_number'].isna()]
filtered_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
19,B00053,2019-10-01 00:05:24,2019-10-01 00:53:03,264.0,264.0,NaN,NaN
155,B00628,2019-10-01 00:34:18,2019-10-01 00:44:55,88.0,229.0,NaN,NaN
161,B00628,2019-10-01 00:51:54,2019-10-01 01:11:21,251.0,88.0,NaN,NaN


In [18]:
filtered_df_null = df_pandas[df_pandas['Affiliated_base_number'].isnull()]
filtered_df_null

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
19,B00053,2019-10-01 00:05:24,2019-10-01 00:53:03,264.0,264.0,NaN,NaN
155,B00628,2019-10-01 00:34:18,2019-10-01 00:44:55,88.0,229.0,NaN,NaN
161,B00628,2019-10-01 00:51:54,2019-10-01 01:11:21,251.0,88.0,NaN,NaN


In fact isnull and isna are identical

In [20]:
from pyspark.sql import types

In [22]:
schema = types.StructType([
    types.StructField('dispatching_base_num',types.StringType(), True),
    types.StructField('pickup_datetime',types.TimestampType(), True),
    types.StructField('dropOff_datetime',types.TimestampType(), True),
    types.StructField('PUlocationID',types.IntegerType(), True),
    types.StructField('DOlocationID',types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [23]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [24]:
df = df.repartition(6)

In [25]:
df.write.parquet('fhv/2019/10/')

In [26]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01717|2019-10-13 23:52:17|2019-10-14 00:00:41|         264|         235|   null|                B01717|
|              B01625|2019-10-02 10:40:00|2019-10-02 11:05:00|         264|         264|   null|                B01625|
|              B01233|2019-10-08 00:09:01|2019-10-08 00:11:14|         264|          81|   null|                B01233|
|              B02735|2019-10-04 07:50:33|2019-10-04 07:58:33|         264|         242|   null|                B02735|
|              B02688|2019-10-02 10:59:00|2019-10-02 11:34:46|         264|         107|   null|                B02688|
|              B01231|2019-10-06 14:39:4

In [28]:
df.createOrReplaceTempView('fhv_data')

In [36]:
spark.sql("""
SELECT
    count(1)
FROM
    fhv_data
WHERE
    date_trunc('day',pickup_datetime)='2019-10-15 00:00:00'
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [38]:
spark.version

'3.3.2'

In [40]:
!ls -l --block-size=M fhv/*/*/*.parquet

-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00000-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00001-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00002-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00003-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00004-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 7M Mar  4 17:24 fhv/2019/10/part-00005-0236b0fe-baaf-45c5-adc7-732d1b1a3dea-c000.snappy.parquet


In [50]:
spark.sql("""
SELECT 
    MAX(CAST(dropoff_datetime AS LONG)-CAST(pickup_datetime AS LONG)) as duration
FROM
fhv_data
""").show()

+----------+
|  duration|
+----------+
|2272149000|
+----------+



In [52]:
spark.sql("""
SELECT 
    MAX(CAST(dropoff_datetime AS LONG)-CAST(pickup_datetime AS LONG))/(60*60) as duration
FROM
fhv_data
""").show()

+--------+
|duration|
+--------+
|631152.5|
+--------+



In [54]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [56]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [57]:
df_zones.write.parquet('zones')

In [58]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [59]:
df_zones.registerTempTable('zones')

In [60]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    fhv_data fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                    
GROUP BY 
    1
ORDER BY
    2 asc
LIMIT 5;
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

